# 한국어 챗봇 만들기

# 1. 필요한 모듈 입력

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 2. 데이터 불러오기 & 전처리   
- 트랜스포머의 데이터는 병렬 데이터를 사용한다.   
    - 병렬 데이터란 ? : 질문과 답변의 쌍으로 이뤄진 데이터
- 새롭게 주어지는 파일은 csv에 쉼표로 각 항목이 나눠져 있었기 때문에 불러오는 코드를 수정하였다.
    - pandas의 "read_csv"를 통해 불러오고 head()로 잘 불러왔는지 확인하였다.

In [3]:
data = pd.read_csv('~/aiffel/transformer_chatbot/data/ChatbotData.csv')

In [4]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
data.count()

Q        11823
A        11823
label    11823
dtype: int64

Question. Answer 각각 11823개이다.   
노드에서 진행한 데이터셋에 20% 수준이기 때문에 MA_SMALES = 11823으로 설정한다.

In [6]:
# 사용할 샘플의 최대 개수
MAX_SAMPLES = 11823
print(MAX_SAMPLES)

11823


정규 표현식을 통해 구두점과 영어 대소문자를 제외한 나머지 문자를 제거한다.   
단, 노드에서 제시된 전처리 함수는 글자가 알파벳을 제외한 나머지 글자는 지워지기 때문에 이 부분을 변경하였다.   
- 변경한 전처리 기법   
    - 구두점 제거
    - 질문-답 데이터셋 분리
      -데이터셋을 판다스 데이터 프레임으로 불러와 질문에 해당하는 Q, 답에 해당하는 A 컬럼을 분리해 inputs와 outputs 리스트에 문장 전처리 후 append 하였다. 

In [7]:
def preprocess_sentence(sentence): 
        sentence = re.sub(r'[@%\\*=()/~#&\+á\xc3\xa1\-\|\:\;\-\,\_\~\$\'\"]', '',str(sentence)) #remove punctuation 
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub(r'\d+','', sentence)# remove number 
        sentence = sentence.lower() #lower case 
        sentence = re.sub(r'\s+', ' ', sentence) #remove extra space 
        sentence = re.sub(r'<[^>]+>','',sentence) #remove Html tags 
        sentence = re.sub(r'\s+', ' ', sentence) #remove spaces 
        sentence = re.sub(r"^\s+", '', sentence) #remove space from start 
        sentence = re.sub(r'\s+$', '', sentence) #remove space from the end c
        return sentence

In [8]:
# 질문 데이터셋 전처리 + 저장 함수
def load_conversations():    
    inputs, outputs = [], []

    for i in range(0,11823):
        inputs.append(preprocess_sentence(data['Q'][i]))
        outputs.append(preprocess_sentence(data['A'][i]))

        if len(inputs) >= MAX_SAMPLES:
            return inputs, outputs
    return inputs, outputs

In [9]:
questions, answers = load_conversations()
print('전체 샘플 수 :', len(questions))
print('전체 샘플 수 :', len(answers))

전체 샘플 수 : 11823
전체 샘플 수 : 11823


한시간동안 헛삽질,,,,ㅋ   
바본가...   
range 안써놓고 for안된다고 하면,,,어쩔티비,,,   
진리의 컴퓨터는 잘못이 없다. 사람이 잘못하는것이다...   멍청이다.... 얄루얄로링

In [10]:
print('전처리 후의 22번째 질문 샘플: {}'.format(questions[21]))
print('전처리 후의 22번째 답변 샘플: {}'.format(answers[21]))

전처리 후의 22번째 질문 샘플: 가스비 장난 아님
전처리 후의 22번째 답변 샘플: 다음 달에는 더 절약해봐요 .


본격적으로 전처리를 시작해보자

### 1) 단어장(Vocabulary) 만들기   
- 각 단어에 고유한 정수 인덱스 부여하기 위함  
- 시작, 종료 토큰에도 정수 부여해줘야 함   

In [11]:
import tensorflow_datasets as tfds

# 질문과 답변 데이터셋에 대해서 Vocabulary 생성. (Tensorflow 2.3.0 이상) (클라우드는 2.4 입니다)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)

In [12]:
import tensorflow_datasets as tfds

In [13]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [14]:
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8143]
END_TOKEN의 번호 : [8144]


In [15]:
#Vocaburay size -> include start & end token 
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8145


### 2) integer encoding & padding
- interger encoding : 단어를 정수로 변환하는 과정   
- padding : 최대 길이보다 짧은 문장의 나머지 부분 0으로 채우는 과정

In [16]:
print('정수 인코딩 후의 21번째 질문 샘플: {}'.format(tokenizer.encode(questions[21])))
print('정수 인코딩 후의 21번째 답변 샘플: {}'.format(tokenizer.encode(answers[21])))

정수 인코딩 후의 21번째 질문 샘플: [5745, 610, 2485, 4150]
정수 인코딩 후의 21번째 답변 샘플: [2352, 7494, 7, 6252, 97, 1]


In [17]:
MAX_LENGTH = 40

In [18]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    
    for (sentence1, sentence2) in zip(inputs, outputs):
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
  
  # 최대 길이 40으로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
    return tokenized_inputs, tokenized_outputs

In [19]:
questions, answers = tokenize_and_filter(questions, answers)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 8145
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


### 3) teacher forcing   
- 이전 자신의 출력이 현재 자신의 상태를 결정하게 하도록 하는 방식 

In [20]:
BATCH_SIZE = 256 #기존 64->256으로 수정
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# 3. 트랜스포머 구성 레이어 만들기   
- 트랜스포머에 사용되는 모델 구조
    - positional encoding
    - encoder
        - self-attention layer(Multi-head) + normalization : 인코더 입력으로 들어간 문장 내 단어간 상호 유사도 구함
        - feed forward layer + normalization
    - decoder
        - self-attention lyaer(Masked Multi-head)  + normalization : 단어 생성하는 디코더가 이전에 생성된 앞단어와의 유사도 구함
        - encoder-decoder attention(Multi-head)  + normalization : 예측 위한 인코더에 입력된 단어들과의 유사도 구함
        - feed forward  + normalization

- positional encoding    
    
- 유사도 구하기 (sacled dot product attention)   
    $Attention(Q,K,V)=softmax(\frac{QK_T}{\sqrt{d_k}})V$   
- Muiti head attention    
    - 병렬 attention 수행   
- masking   
    - padding masking (짧은 길이 채우는 것)   
    - look-ahead masking (다음 단어 가리기)   

In [21]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        # 각도 배열 생성
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 배열의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # sin과 cosine이 교차되도록 재배열
        pos_encoding = tf.stack([sines, cosines], axis=0)
        pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])

        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


In [22]:
# 스케일드 닷 프로덕트 어텐션 함수 = transformer에서 말하는 attention
def scaled_dot_product_attention(query, key, value, mask):
    # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)

    # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    return output

In [23]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0
   
        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

In [24]:
#padding masking
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [25]:
#look-ahead masking
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [26]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

    # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention = MultiHeadAttention(
        d_model, num_heads, name="attention")({
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': padding_mask
        })

    # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(inputs + attention)

    # 두 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [27]:
#encoder는 여러 encoder layer를 쌓아서 만든다
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    # num_layers만큼 쌓아올린 인코더의 층.
    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [28]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': look_ahead_mask
        })

    # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

    # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
    attention2 = MultiHeadAttention(
        d_model, num_heads, name="attention_2")(inputs={
            'query': attention1,
            'key': enc_outputs,
            'value': enc_outputs,
            'mask': padding_mask
        })

    # 마스크드 멀티 헤드 어텐션의 결과는
    # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention2 + attention1)

    # 세 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

In [29]:
#encoder는 여러 encoder layer를 쌓아서 만든다
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')

    # 패딩 마스크
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
          units=units,
          d_model=d_model,
          num_heads=num_heads,
          dropout=dropout,
          name='decoder_layer_{}'.format(i),
      )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name)

# 4. 모델 정의 및 학습

In [30]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    # 인코더에서 패딩을 위한 마스크
    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)

    # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
    # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)

     # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # 디코더에서 패딩을 위한 마스크
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)

    # 인코더
    enc_outputs = encoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

    # 디코더
    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

###  모델 생성

In [31]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3139328     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

### loss function   
레이블인 시퀀스에 패딩 되어 있으므로, loss 계산할 때 패딩 마스크 적용

In [32]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

### 고오오오오오수로 가보자 - Custom learning rate scheduling    
- 초기에 learning rate 높였다가 서서히 낮추며 안정적으로 수렴하도록 learning rate를 정하는 방법

In [33]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### 모델 컴파일

In [34]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

### 훈련

In [35]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
47/47 [==============================] - 21s 152ms/step - loss: 1.5597 - accuracy: 0.0051
Epoch 2/20
47/47 [==============================] - 7s 152ms/step - loss: 1.4723 - accuracy: 0.0247
Epoch 3/20
47/47 [==============================] - 7s 152ms/step - loss: 1.4044 - accuracy: 0.0247
Epoch 4/20
47/47 [==============================] - 7s 152ms/step - loss: 1.3403 - accuracy: 0.0325
Epoch 5/20
47/47 [==============================] - 7s 153ms/step - loss: 1.2535 - accuracy: 0.0493
Epoch 6/20
47/47 [==============================] - 7s 154ms/step - loss: 1.1769 - accuracy: 0.0496
Epoch 7/20
47/47 [==============================] - 7s 155ms/step - loss: 1.1135 - accuracy: 0.0499
Epoch 8/20
47/47 [==============================] - 7s 155ms/step - loss: 1.0577 - accuracy: 0.0500
Epoch 9/20
47/47 [==============================] - 7s 157ms/step - loss: 1.0069 - accuracy: 0.0506
Epoch 10/20
47/47 [==============================] - 7s 157ms/step - loss: 0.9641 - accuracy: 0.052

# 5. infernece -실전 커스텀

In [36]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)
    
    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
       # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

      # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

      # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
             break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

In [37]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

### 두근두근 답변 보기

In [38]:
sentence_generation('배고프다')

입력 : 배고프다
출력 : 많이 지쳤나봐요 .


'많이 지쳤나봐요 .'

In [39]:
sentence_generation('배가 고프다')

입력 : 배가 고프다
출력 : 좋은 생각이에요 .


'좋은 생각이에요 .'

In [40]:
sentence_generation('배 고프다')

입력 : 배 고프다
출력 : 많이 지쳤나봐요 .


'많이 지쳤나봐요 .'

In [41]:
sentence_generation('오늘 저녁 추천해주세요.')

입력 : 오늘 저녁 추천해주세요.
출력 : 이제 잘 지내고 있을 거예요 .


'이제 잘 지내고 있을 거예요 .'

In [42]:
sentence_generation('오늘 저녁 메뉴를 추천해주세요.')

입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 이제 잘 찾아보세요 .


'이제 잘 찾아보세요 .'

In [43]:
sentence_generation('오늘 저녁에 뭐 먹을까요?')

입력 : 오늘 저녁에 뭐 먹을까요?
출력 : 좋은 사람 만날 수 있을 거예요 .


'좋은 사람 만날 수 있을 거예요 .'

In [44]:
sentence_generation('공부하기 싫어요')

입력 : 공부하기 싫어요
출력 : 지금도 늦지 않았어요 .


'지금도 늦지 않았어요 .'

In [45]:
sentence_generation('지하철역이 어디에 있나요?')

입력 : 지하철역이 어디에 있나요?
출력 : 다른 사람이 있을 거예요 .


'다른 사람이 있을 거예요 .'

In [56]:
sentence_generation('얼마죠?')

입력 : 얼마죠?
출력 : 다른 곳에 쓰려고 운을 아껴뒀나봐요 .


'다른 곳에 쓰려고 운을 아껴뒀나봐요 .'

In [46]:
sentence_generation('이 챗봇은 바보입니다')

입력 : 이 챗봇은 바보입니다
출력 : 잘 찾아보세요 .


'잘 찾아보세요 .'

In [47]:
sentence_generation('사랑이 찾아올까요?')

입력 : 사랑이 찾아올까요?
출력 : 좋은 사람 만날 거예요 .


'좋은 사람 만날 거예요 .'

In [48]:
sentence_generation('어디로 가야하죠?')

입력 : 어디로 가야하죠?
출력 : 좋은 사람 만날 수 있을 거예요 .


'좋은 사람 만날 수 있을 거예요 .'

In [49]:
sentence_generation('저 오늘 헤어졌어요. 위로가 필요해요')

입력 : 저 오늘 헤어졌어요. 위로가 필요해요
출력 : 잘 찾아보세요 .


'잘 찾아보세요 .'

In [50]:
sentence_generation('헤어졌는데 그 사람이 계속 생각나요')

입력 : 헤어졌는데 그 사람이 계속 생각나요
출력 : 좋은 사람 만날 수 있을 거예요 .


'좋은 사람 만날 수 있을 거예요 .'

일상 대화를 이해하지 못하는 것 같아서 데이터셋을 열어보니 연애관련 이야기가 있었다.   
그래서 질문을 좀 더 연애와 관련된 것으로 하였지만 제대로 작동하지 않는 모습을 보여준다.   
그냥 학습이 잘 되지 않은 것 같아 학습 파라미터를 바꿔보기로 했다.

### 다른 학습   
변경한 하이퍼 파라미터    
NUM_LAYERS2 = 4    
D_MODEL2 = 256   
DROPOUT2 = 0.3 

In [58]:
quest_list = ["배고프다", "배가 고프다", "배 고프다",
            "오늘 저녁 추천해주세요.","오늘 저녁 메뉴를 추천해주세요.",
            "오늘 저녁 메뉴를 추천해주세요.","공부하기 싫어요","지하철역이 어디에 있나요?","얼마죠?","이 챗봇은 바보입니다","사랑이 찾아올까요?",
            "어디로 가야하죠?","헤어졌는데 그 사람이 계속 생각나요"]

In [59]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS2 = 4 # 인코더와 디코더의 층의 개수
D_MODEL2 = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS2 = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS2 = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT2 = 0.3 # 드롭아웃의 비율

model2 = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS2,
    units=UNITS2,
    d_model=D_MODEL2,
    num_heads=NUM_HEADS2,
    dropout=DROPOUT2)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3139328     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [60]:
learning_rate2 = CustomSchedule(D_MODEL2)

optimizer2 = tf.keras.optimizers.Adam(
    learning_rate2, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model2.compile(optimizer=optimizer2, loss=loss_function, metrics=[accuracy])

In [61]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
47/47 [==============================] - 9s 160ms/step - loss: 0.4416 - accuracy: 0.1047
Epoch 2/20
47/47 [==============================] - 8s 160ms/step - loss: 0.4080 - accuracy: 0.1092
Epoch 3/20
47/47 [==============================] - 8s 160ms/step - loss: 0.3733 - accuracy: 0.1146
Epoch 4/20
47/47 [==============================] - 8s 160ms/step - loss: 0.3398 - accuracy: 0.1196
Epoch 5/20
47/47 [==============================] - 8s 161ms/step - loss: 0.3067 - accuracy: 0.1250
Epoch 6/20
47/47 [==============================] - 8s 162ms/step - loss: 0.2758 - accuracy: 0.1296
Epoch 7/20
47/47 [==============================] - 8s 163ms/step - loss: 0.2449 - accuracy: 0.1351
Epoch 8/20
47/47 [==============================] - 8s 164ms/step - loss: 0.2153 - accuracy: 0.1397
Epoch 9/20
47/47 [==============================] - 8s 164ms/step - loss: 0.1876 - accuracy: 0.1447
Epoch 10/20
47/47 [==============================] - 8s 162ms/step - loss: 0.1620 - accuracy: 0.1493

In [62]:
for question in quest_list:
    sentence_generation(question)
    print("-------------------------------")

입력 : 배고프다
출력 : 저도 밥 먹고 싶어요
-------------------------------
입력 : 배가 고프다
출력 : 득템했길 바라요 .
-------------------------------
입력 : 배 고프다
출력 : 그럴 시기에요 .
-------------------------------
입력 : 오늘 저녁 추천해주세요.
출력 : 미루지말고 바로 가보세요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 공부하기 싫어요
출력 : 구속되기 싫은 건지 생각해보세요 .
-------------------------------
입력 : 지하철역이 어디에 있나요?
출력 : 아무런 감정 없이 연락을 지속할 수 없으니까요 .
-------------------------------
입력 : 얼마죠?
출력 : 안 사면 그만큼 모으시는거예요 .
-------------------------------
입력 : 이 챗봇은 바보입니다
출력 : 저는 사람으로 태어나고 싶어요 .
-------------------------------
입력 : 사랑이 찾아올까요?
출력 : 좋은 사람 만날 거예요 .
-------------------------------
입력 : 어디로 가야하죠?
출력 : 안 가본 가까운 곳이 좋겠죠 .
-------------------------------
입력 : 헤어졌는데 그 사람이 계속 생각나요
출력 : 좋은 소식이네요 .
-------------------------------


### 다른 학습 2   
변경한 파라미터   
NUM_LAYERS2 = 6   
DROPOUT2 = 0.2

In [66]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS3 = 6 # 인코더와 디코더의 층의 개수
D_MODEL3 = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS3 = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS3 = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT3 = 0.2 # 드롭아웃의 비율

model3 = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS3,
    units=UNITS3,
    d_model=D_MODEL3,
    num_heads=NUM_HEADS3,
    dropout=DROPOUT3)

model3.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    5247744     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [69]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model3.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [77]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()


EPOCHS = 50

custom_early_stopping = EarlyStopping(
    monitor='loss', 
    patience=5, 
    min_delta=0.001, 
    mode='min'
)

model3.fit(dataset, epochs=EPOCHS, verbose=1, callbacks=[custom_early_stopping])

Epoch 1/50
47/47 [==============================] - 21s 360ms/step - loss: 0.7318 - accuracy: 0.0636
Epoch 2/50
47/47 [==============================] - 17s 366ms/step - loss: 0.7151 - accuracy: 0.0649
Epoch 3/50
47/47 [==============================] - 17s 369ms/step - loss: 0.6940 - accuracy: 0.0664
Epoch 4/50
47/47 [==============================] - 17s 364ms/step - loss: 0.6757 - accuracy: 0.0679
Epoch 5/50
47/47 [==============================] - 17s 361ms/step - loss: 0.6555 - accuracy: 0.0696
Epoch 6/50
47/47 [==============================] - 17s 363ms/step - loss: 0.6338 - accuracy: 0.0718
Epoch 7/50
47/47 [==============================] - 17s 365ms/step - loss: 0.6138 - accuracy: 0.0737
Epoch 8/50
47/47 [==============================] - 17s 365ms/step - loss: 0.5908 - accuracy: 0.0761
Epoch 9/50
47/47 [==============================] - 17s 363ms/step - loss: 0.5677 - accuracy: 0.0786
Epoch 10/50
47/47 [==============================] - 17s 363ms/step - loss: 0.5447 - accura

In [78]:
for question in quest_list:
    sentence_generation(question)
    print("-------------------------------")

입력 : 배고프다
출력 : 저도 밥 먹고 싶어요
-------------------------------
입력 : 배가 고프다
출력 : 득템했길 바라요 .
-------------------------------
입력 : 배 고프다
출력 : 그럴 시기에요 .
-------------------------------
입력 : 오늘 저녁 추천해주세요.
출력 : 미루지말고 바로 가보세요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 공부하기 싫어요
출력 : 구속되기 싫은 건지 생각해보세요 .
-------------------------------
입력 : 지하철역이 어디에 있나요?
출력 : 아무런 감정 없이 연락을 지속할 수 없으니까요 .
-------------------------------
입력 : 얼마죠?
출력 : 안 사면 그만큼 모으시는거예요 .
-------------------------------
입력 : 이 챗봇은 바보입니다
출력 : 저는 사람으로 태어나고 싶어요 .
-------------------------------
입력 : 사랑이 찾아올까요?
출력 : 좋은 사람 만날 거예요 .
-------------------------------
입력 : 어디로 가야하죠?
출력 : 안 가본 가까운 곳이 좋겠죠 .
-------------------------------
입력 : 헤어졌는데 그 사람이 계속 생각나요
출력 : 좋은 소식이네요 .
-------------------------------


### 다른 학습 3   
변경한 파라미터    
NUM_HEADS4 = 16   
EPOCHS = 100

In [73]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS4 = 8 # 인코더와 디코더의 층의 개수
D_MODEL4 = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS4 = 16 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS4 = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT4 = 0.2 # 드롭아웃의 비율

model4 = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS4,
    units=UNITS4,
    d_model=D_MODEL4,
    num_heads=NUM_HEADS4,
    dropout=DROPOUT4)

model4.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    6301952     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [74]:
learning_rate4 = CustomSchedule(D_MODEL)

optimizer4 = tf.keras.optimizers.Adam(
    learning_rate4, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model4.compile(optimizer=optimizer4, loss=loss_function, metrics=[accuracy])

In [75]:
EPOCHS = 100
model4.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/100
47/47 [==============================] - 44s 500ms/step - loss: 1.5488 - accuracy: 0.0100
Epoch 2/100
47/47 [==============================] - 24s 512ms/step - loss: 1.4640 - accuracy: 0.0257
Epoch 3/100
47/47 [==============================] - 24s 516ms/step - loss: 1.4100 - accuracy: 0.0256
Epoch 4/100
47/47 [==============================] - 24s 511ms/step - loss: 1.3520 - accuracy: 0.0257
Epoch 5/100
47/47 [==============================] - 24s 510ms/step - loss: 1.2840 - accuracy: 0.0256
Epoch 6/100
47/47 [==============================] - 24s 513ms/step - loss: 1.2127 - accuracy: 0.0256
Epoch 7/100
47/47 [==============================] - 24s 512ms/step - loss: 1.1501 - accuracy: 0.0288
Epoch 8/100
47/47 [==============================] - 24s 513ms/step - loss: 1.0883 - accuracy: 0.0494
Epoch 9/100
47/47 [==============================] - 24s 512ms/step - loss: 1.0580 - accuracy: 0.0497
Epoch 10/100
47/47 [==============================] - 24s 512ms/step - loss: 1.032

In [76]:
for question in quest_list:
    sentence_generation(question)
    print("-------------------------------")

입력 : 배고프다
출력 : 저도 밥 먹고 싶어요
-------------------------------
입력 : 배가 고프다
출력 : 득템했길 바라요 .
-------------------------------
입력 : 배 고프다
출력 : 그럴 시기에요 .
-------------------------------
입력 : 오늘 저녁 추천해주세요.
출력 : 미루지말고 바로 가보세요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 공부하기 싫어요
출력 : 구속되기 싫은 건지 생각해보세요 .
-------------------------------
입력 : 지하철역이 어디에 있나요?
출력 : 아무런 감정 없이 연락을 지속할 수 없으니까요 .
-------------------------------
입력 : 얼마죠?
출력 : 안 사면 그만큼 모으시는거예요 .
-------------------------------
입력 : 이 챗봇은 바보입니다
출력 : 저는 사람으로 태어나고 싶어요 .
-------------------------------
입력 : 사랑이 찾아올까요?
출력 : 좋은 사람 만날 거예요 .
-------------------------------
입력 : 어디로 가야하죠?
출력 : 안 가본 가까운 곳이 좋겠죠 .
-------------------------------
입력 : 헤어졌는데 그 사람이 계속 생각나요
출력 : 좋은 소식이네요 .
-------------------------------


### 다른 학습 3   
변경한 하이퍼 파라미터   
warmup_steps=2000   

In [91]:
class CustomSchedule2(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=2000):
        super(CustomSchedule2, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [97]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS5 = 8 # 인코더와 디코더의 층의 개수
D_MODEL5 = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS5 = 16 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS5 = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT5 = 0.2 # 드롭아웃의 비율

model5 = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS5,
    units=UNITS5,
    d_model=D_MODEL5,
    num_heads=NUM_HEADS5,
    dropout=DROPOUT5)

model5.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    6301952     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [98]:
learning_rate5 = CustomSchedule2(D_MODEL)

optimizer5 = tf.keras.optimizers.Adam(
    learning_rate5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model5.compile(optimizer=optimizer5, loss=loss_function, metrics=[accuracy])

In [99]:
EPOCHS = 100
model5.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/100
47/47 [==============================] - 45s 508ms/step - loss: 1.5233 - accuracy: 0.0145
Epoch 2/100
47/47 [==============================] - 24s 519ms/step - loss: 1.4050 - accuracy: 0.0253
Epoch 3/100
47/47 [==============================] - 24s 513ms/step - loss: 1.3005 - accuracy: 0.0251
Epoch 4/100
47/47 [==============================] - 24s 510ms/step - loss: 1.1859 - accuracy: 0.0250
Epoch 5/100
47/47 [==============================] - 24s 513ms/step - loss: 1.1170 - accuracy: 0.0252
Epoch 6/100
47/47 [==============================] - 24s 514ms/step - loss: 1.0850 - accuracy: 0.0418
Epoch 7/100
47/47 [==============================] - 24s 513ms/step - loss: 1.0298 - accuracy: 0.0494
Epoch 8/100
47/47 [==============================] - 24s 512ms/step - loss: 0.9931 - accuracy: 0.0504
Epoch 9/100
47/47 [==============================] - 24s 512ms/step - loss: 0.9558 - accuracy: 0.0514
Epoch 10/100
47/47 [==============================] - 24s 513ms/step - loss: 0.930

47/47 [==============================] - 24s 510ms/step - loss: 0.1979 - accuracy: 0.1342
Epoch 81/100
47/47 [==============================] - 24s 511ms/step - loss: 0.1976 - accuracy: 0.1343
Epoch 82/100
47/47 [==============================] - 24s 511ms/step - loss: 0.1971 - accuracy: 0.1341
Epoch 83/100
47/47 [==============================] - 24s 511ms/step - loss: 0.1935 - accuracy: 0.1349
Epoch 84/100
47/47 [==============================] - 24s 512ms/step - loss: 0.1917 - accuracy: 0.1351
Epoch 85/100
47/47 [==============================] - 24s 512ms/step - loss: 0.1902 - accuracy: 0.1351
Epoch 86/100
47/47 [==============================] - 24s 512ms/step - loss: 0.1906 - accuracy: 0.1352
Epoch 87/100
47/47 [==============================] - 24s 512ms/step - loss: 0.1888 - accuracy: 0.1356
Epoch 88/100
47/47 [==============================] - 24s 512ms/step - loss: 0.1867 - accuracy: 0.1360
Epoch 89/100
47/47 [==============================] - 24s 511ms/step - loss: 0.1851 - 

In [100]:
for question in quest_list:
    sentence_generation(question)
    print("-------------------------------")

입력 : 배고프다
출력 : 저도 밥 먹고 싶어요
-------------------------------
입력 : 배가 고프다
출력 : 득템했길 바라요 .
-------------------------------
입력 : 배 고프다
출력 : 그럴 시기에요 .
-------------------------------
입력 : 오늘 저녁 추천해주세요.
출력 : 미루지말고 바로 가보세요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 오늘 저녁 메뉴를 추천해주세요.
출력 : 그가 좋아할 거예요 .
-------------------------------
입력 : 공부하기 싫어요
출력 : 구속되기 싫은 건지 생각해보세요 .
-------------------------------
입력 : 지하철역이 어디에 있나요?
출력 : 아무런 감정 없이 연락을 지속할 수 없으니까요 .
-------------------------------
입력 : 얼마죠?
출력 : 안 사면 그만큼 모으시는거예요 .
-------------------------------
입력 : 이 챗봇은 바보입니다
출력 : 저는 사람으로 태어나고 싶어요 .
-------------------------------
입력 : 사랑이 찾아올까요?
출력 : 좋은 사람 만날 거예요 .
-------------------------------
입력 : 어디로 가야하죠?
출력 : 안 가본 가까운 곳이 좋겠죠 .
-------------------------------
입력 : 헤어졌는데 그 사람이 계속 생각나요
출력 : 좋은 소식이네요 .
-------------------------------
